<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning_LANA/blob/master/keras/keras_Titanic_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Antes de executar este notebook, crie um ambiente virtual com o comando abaixo no terminal:

conda create -n exercicioscap02 numpy pandas scikit-learn keras tensorflow

In [1]:
!nvidia-smi

Tue Jan 15 16:16:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    63W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Construindo uma Rede Neural - Titanic

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

In [0]:
np.random.seed(1337)

In [4]:
# leitura dos dados
from google.colab import files
uploaded = files.upload()

Saving titanic_gender_submission.csv to titanic_gender_submission.csv
Saving titanic_test.csv to titanic_test.csv
Saving titanic_train.csv to titanic_train.csv
Saving titanic.csv to titanic.csv
Saving titanicOLD.csv to titanicOLD.csv
Saving mnist.csv to mnist.csv


In [5]:
!ls

mnist.csv    titanic.csv		    titanicOLD.csv    titanic_train.csv
sample_data  titanic_gender_submission.csv  titanic_test.csv


In [0]:
#import io
#df2 = pd.read_csv(io.StringIO(uploaded['Filename.csv']))

In [0]:
# folder_titan = '/media/datasets/DeepLearningI/Cap02/titanic.csv'
df = pd.read_csv('titanic.csv')

In [7]:
df.head()

,Survived,Class,Sex,Age,Fare
0,0,3,1,22.0,7.2500
1,1,1,0,38.0,71.2833
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500


In [8]:
len(df)

891

In [9]:
df_train = df.iloc[:712, :]

df_train.head()

,Survived,Class,Sex,Age,Fare
0,0,3,1,22.0,7.2500
1,1,1,0,38.0,71.2833
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500


In [0]:
scaler = StandardScaler()
features = ['Class', 'Sex', 'Age', 'Fare']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [11]:
X_train[:5]

array([[ 0.83290956,  0.74926865, -0.61259594, -0.51933199],
       [-1.55353553, -1.33463478,  0.6184268 ,  0.79718222],
       [ 0.83290956, -1.33463478, -0.30484025, -0.5054541 ],
       [-1.55353553, -1.33463478,  0.38761004,  0.42333654],
       [ 0.83290956,  0.74926865,  0.38761004, -0.50288412]])

In [12]:
y_train_onehot[:3]

array([[1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [0]:
df_test = df.iloc[712:, :]

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

In [14]:
np.set_printoptions(precision=5, suppress=True)
X_train[:5]

array([[ 0.83291,  0.74927, -0.6126 , -0.51933],
       [-1.55354, -1.33463,  0.61843,  0.79718],
       [ 0.83291, -1.33463, -0.30484, -0.50545],
       [-1.55354, -1.33463,  0.38761,  0.42334],
       [ 0.83291,  0.74927,  0.38761, -0.50288]])

In [15]:
y_train[:5]

array([0, 1, 1, 1, 0])

## Criando um Modelo de Árvore de Decisão ( Benchmark)

In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['Survived'].values)

y_prediction = model.predict(X_test)
print ("\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

Acurácia 0.8100558659217877


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## Rede Neural com 1 Camada

In [17]:
print(features)
X_train[:3]

['Class', 'Sex', 'Age', 'Fare']


array([[ 0.83291,  0.74927, -0.6126 , -0.51933],
       [-1.55354, -1.33463,  0.61843,  0.79718],
       [ 0.83291, -1.33463, -0.30484, -0.50545]])

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()

model.add(Dense(input_dim=4, units=2)) # saída - units

model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\nTempo gasto: %s segundos' % str(time() - start))

Using TensorFlow backend.


Epoch 1/1
712/712 [==============================] - 2s 3ms/step - loss: 1.4410 - acc: 0.3413

Tempo gasto: 2.3684346675872803 segundos


In [19]:
y_prediction = model.predict_classes(X_test)
print ("\n\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))



Acurácia 0.3240223463687151


## Rede Neural com 2 Camadas

In [20]:
start = time()

model = Sequential()

model.add(Dense(input_dim=4, units = 100)) # units - numero de neuronios

model.add(Dense(units=2))

model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\nTempo gasto: %s segundos' % str(time() - start))

Epoch 1/1
712/712 [==============================] - 0s 261us/step - loss: 0.6605 - acc: 0.6517

Tempo gasto: 0.4491574764251709 segundos


In [21]:
y_prediction = model.predict_classes(X_test)
print ("\n\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))



Acurácia 0.8100558659217877


## Rede Neural com 3 Camadas

In [22]:
start = time()

model = Sequential()

model.add(Dense(input_dim=4, units=100))

model.add(Dense(units=100))

model.add(Dense(units=2))

model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\nTempo gasto: %s segundos' % str(time() - start))

Epoch 1/1
712/712 [==============================] - 0s 328us/step - loss: 0.7281 - acc: 0.5239

Tempo gasto: 0.5178258419036865 segundos


In [23]:
y_prediction = model.predict_classes(X_test)
print ("\n\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))



Acurácia 0.776536312849162
